Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_fail_sum">_fail_sum</a> theorem
========

In [ ]:
import proveit
theory = proveit.Theory() # the theorem's theory
from proveit import a, b, c, e, k, l, m, n, x, y, N, R, S, X, Psi, defaults, fx, gy
from proveit.logic import Equals, Forall, InSet, superset_eq
from proveit.numbers import (
    zero, one, two, Add, Complex, Integer, Less, LessEq, NaturalPos, Neg, Real, subtract)
from proveit.numbers.negation import negated_weak_bound
from proveit.numbers.modular import full_modular_range_equiv_left_shift
from proveit.numbers.summation import equiv_sums
from proveit.physics.quantum import register_qubit_all_probs, register_qubit_born_rule
from proveit.physics.quantum.QPE import (
    _alpha_def, _b, _best_is_int, _fail_def, _full_domain, _m, ModAdd, _mod_add_def, _neg_domain,
    _phase_est, _pos_domain, _Psi, _Psi_ket_in_qreg_space,
    _success_prob_is_real, _success_sum, _t, _t_in_natural_pos, _two_pow_t, _two_pow_t_is_nat_pos,
    _two_pow_t_minus_one, _two_pow_t_minus_one_is_nat_pos)
from proveit.statistics import Prob, prob_in_real

In [ ]:
%proving _fail_sum

In [ ]:
e_in_domain = _fail_sum.all_conditions()[0]

In [ ]:
defaults.assumptions = _fail_sum.all_conditions()

In [ ]:
_two_pow_t_is_nat_pos

In [ ]:
_two_pow_t_minus_one_is_nat_pos

In [ ]:
e_in_domain.derive_element_in_integer()

In [ ]:
e_in_domain.derive_element_lower_bound()

In [ ]:
InSet(e, NaturalPos).prove()

In [ ]:
_fail_def

In [ ]:
fail_def_inst = _fail_def.instantiate()

In [ ]:
_success_prob_is_real

In [ ]:
success_prob_is_real_inst = _success_prob_is_real.instantiate()

In [ ]:
_success_sum

In [ ]:
success_sum_1 = _success_sum.instantiate()

In [ ]:
# probabilities are Real
prob_in_real

In [ ]:
# instantiate that for our specific type of probabilities being summed;
# then in the next cell we can conclude the summation itself is real
prob_in_real_inst = prob_in_real.instantiate(
        {x: _m, X: success_sum_1.rhs.summand.operands[0]})

In [ ]:
# testing
success_sum_1.rhs.summand.operands[0]

In [ ]:
# modified from cell above to use _phase_est instead of _m
# instantiate that for our specific type of probabilities being summed;
# then in the next cell we can conclude the summation itself is real
prob_in_real_inst = prob_in_real.instantiate(
        {x: _phase_est, X: success_sum_1.rhs.summand.operands[0]})

In [ ]:
success_sum_1.rhs.deduce_in_number_set(Real)

In [ ]:
fail_def_inst.rhs

In [ ]:
# fail_def_inst.rhs.deduce_bound([success_sum_1])

In [ ]:
# assert False

In [ ]:
success_sum_2 = success_sum_1.derive_negated().with_direction_reversed()

In [ ]:
success_sum_3 = success_sum_2.derive_shifted(one, addend_side='left')

In [ ]:
success_sum_4 = success_sum_3.inner_expr().lhs.with_subtraction_at(one)

In [ ]:
success_sum_5 = success_sum_4.inner_expr().rhs.with_subtraction_at(one)

In [ ]:
fail_sum_1 = fail_def_inst.sub_left_side_into(success_sum_5)

In [ ]:
register_qubit_all_probs

In [ ]:
# we need to know that t_ is in NaturalPos for the instantiation to work:
_t_in_natural_pos

In [ ]:
# and we need to know that |Psi> is in the right domain:
# PsiKetInQRegSpace
_Psi_ket_in_qreg_space

In [ ]:
# old/previous -- replace with … _phase_est_from_meas? See below
# and need to know that m_ is the measure of ket |Psi>
# _m_def

In [ ]:
# testing
from proveit.physics.quantum.QPE import _phase_est_from_meas
_phase_est_from_meas

In [ ]:
_scaled_phase_est = _phase_est_from_meas.lhs

In [ ]:
# previously; replaced with new instantiation below
# register_qubit_all_probs.instantiate({n: t_, Psi: Psi_, m: m_})

In [ ]:
# new?
register_qubit_all_probs.instantiate({n: _t, Psi: _Psi, m: _scaled_phase_est})

In [ ]:
full_modular_range_equiv_left_shift

In [ ]:
lower_bound = Add(Neg(_two_pow_t_minus_one), one)

In [ ]:
upper_bound = _two_pow_t_minus_one

For an instantiation further below, we will need to know that

(2^{t - 1} - (-2^{t - 1} + 1)) = (2^{t} - 1)

In [ ]:
bound_diff_eq_1 = subtract(upper_bound, lower_bound).simplification()

In [ ]:
bound_diff_eq_2 = bound_diff_eq_1.inner_expr().rhs.operands[0].combine_exponents()

In [ ]:
bound_diff_eq_3 = bound_diff_eq_2.inner_expr().rhs.with_subtraction_at(one)

In [ ]:
# for the instantiation below, need to know that _b is an Integer
_best_is_int

In [ ]:
range_equiv_1 = full_modular_range_equiv_left_shift.instantiate(
        {N: _two_pow_t, a: lower_bound, b: upper_bound, c: _b, x: l})

In [ ]:
# recall our definition of modular addition
# (the t here is _t, the size of the first register)
_mod_add_def

In [ ]:
# for the instantiation, we need to know that ell is an Integer:
l_in_domain = range_equiv_1.lhs.all_conditions()[0]
l_in_domain.derive_element_in_integer(assumptions=[l_in_domain])

In [ ]:
b_mod_add_l_def = _mod_add_def.instantiate({a:_b, b:l}, assumptions=[l_in_domain])

In [ ]:
temp_for_instance_substitution = b_mod_add_l_def.derive_reversed().generalize(l, domain=l_in_domain.domain)

In [ ]:
set_of_all_mod_add_equiv = range_equiv_1.lhs.instance_substitution(temp_for_instance_substitution)

In [ ]:
range_equiv = set_of_all_mod_add_equiv.sub_right_side_into(range_equiv_1)

In [ ]:
# from the numbers.summation pkg
# what about the mapping from S to R? Must it be one-to-one? Maybe we could add
# a condition so that |S| = |R| ? Maybe both finite? Both set |S| = n, for example?
# If they are infinite then we still have a problem — Bob
# for now, we can restrict to finite sets
equiv_sums

In [ ]:
equiv_sums_inst = equiv_sums.instantiate(
        {fx: Prob(Equals(_m, x), _m),
         gy: ModAdd(_b, y),
         R: range_equiv.rhs,
         S: range_equiv.lhs.domain,
         x: k,
         y: l})

In [ ]:
# this helps the next step where we want to show Pr_m[m=a] is Complex
equiv_sums_inst.antecedent.instance_expr.element.deduce_in_real()

In [ ]:
from proveit import ExprTuple
InSet(equiv_sums_inst.antecedent.instance_expr.element, Complex).prove().generalize([[a]], [[equiv_sums_inst.antecedent.domain]])

In [ ]:
# now we can derive the consequent from equiv_sums_inst:
shifted_all_prob_equals_one = equiv_sums_inst.derive_consequent().derive_reversed()

In [ ]:
# Recall fail_sum_1 from earlier
fail_sum_1

In [ ]:
fail_sum_2 = shifted_all_prob_equals_one.sub_left_side_into(fail_sum_1)

Now some work to establish that $-2^{t-1} + 1 \le \epsilon \le 2^{t-1}$, so that we can eventually partition that first summation at index = $\epsilon$.

In [ ]:
# eps_upper_bound = eps_in_domain.derive_element_upper_bound()

In [ ]:
e_upper_bound = e_in_domain.derive_element_upper_bound()

In [ ]:
# Less(eps_upper_bound.rhs, Add(eps_upper_bound.rhs, two)).prove()

In [ ]:
Less(e_upper_bound.rhs, Add(e_upper_bound.rhs, two)).prove()

In [ ]:
# Less(eps_upper_bound.lhs, Add(eps_upper_bound.rhs, two)).prove().simplify()

In [ ]:
Less(e_upper_bound.lhs, Add(e_upper_bound.rhs, two)).prove().simplify()

In [ ]:
Less(Neg(_two_pow_t_minus_one), zero).prove()

In [ ]:
Less(Neg(_two_pow_t_minus_one), zero).derive_shifted(one)

In [ ]:
# eps_in_domain.derive_element_lower_bound()

In [ ]:
e_in_domain.derive_element_lower_bound()

In [ ]:
# Less(Add(Neg(two_pow_t_minus_one), one), eps_upper_bound.lhs).prove()

In [ ]:
Less(Add(Neg(_two_pow_t_minus_one), one), e_upper_bound.lhs).prove()

Now, we're ready to partition our summation at the index value $\epsilon$.

In [ ]:
# shifted_all_prob_split_1 = shifted_all_prob_equals_one.lhs.partition(eps, side='after')

In [ ]:
shifted_all_prob_split_1 = shifted_all_prob_equals_one.lhs.partition(e, side='after')

And now we do some more work before being able to split again …

In [ ]:
# Less(Neg(eps), zero).prove()

In [ ]:
Less(Neg(e), zero).prove()

In [ ]:
# Less(Neg(eps), eps).prove()

In [ ]:
Less(Neg(e), e).prove()

In [ ]:
# upper_bound_add_one = Add(eps_upper_bound.rhs, one)

In [ ]:
upper_bound_add_one = Add(e_upper_bound.rhs, one)

In [ ]:
# Less(eps_upper_bound.rhs, upper_bound_add_one).prove()

In [ ]:
Less(e_upper_bound.rhs, upper_bound_add_one).prove()

In [ ]:
# Less(eps, upper_bound_add_one).prove()

In [ ]:
Less(e, upper_bound_add_one).prove()

In [ ]:
# Less(Neg(one), zero).prove()
# Less(eps, upper_bound_add_one).left_mult_both_sides(Neg(one)).inner_expr().rhs.distribute()

In [ ]:
Less(Neg(one), zero).prove()
Less(e, upper_bound_add_one).left_mult_both_sides(Neg(one)).inner_expr().rhs.distribute()

In [ ]:
# shifted_all_prob_split_1_lhs_sum_split = shifted_all_prob_split_1.rhs.operands[0].partition(Neg(eps), side='before')

In [ ]:
shifted_all_prob_split_1_lhs_sum_split = shifted_all_prob_split_1.rhs.operands[0].partition(Neg(e), side='before')

In [ ]:
# recall:
shifted_all_prob_split_1

In [ ]:
shifted_all_prob_split_2 = shifted_all_prob_split_1_lhs_sum_split.sub_right_side_into(shifted_all_prob_split_1)

In [ ]:
shifted_all_prob_split_3 = shifted_all_prob_split_2.inner_expr().rhs.disassociate(0)

In [ ]:
# recall:
fail_sum_2

In [ ]:
fail_sum_3 = shifted_all_prob_split_3.sub_right_side_into(fail_sum_2)

In [ ]:
fail_sum_4 = fail_sum_3.inner_expr().rhs.disassociate(0)

In [ ]:
register_qubit_born_rule

In [ ]:
l_in_domain

In [ ]:
# recall (and recall we have already proven that l is an Integer)
l_in_domain

In [ ]:
# recall:
b_mod_add_l_def

In [ ]:
InSet(Add(b_, l), Integer).prove(assumptions=[l_in_domain])

In [ ]:
two_pow_t_is_nat_pos

In [ ]:
b_add_l_mod_2t_in_non_neg_domain = b_mod_add_l_def.rhs.deduce_in_interval(assumptions=[l_in_domain])

In [ ]:
b_mod_add_l_def.sub_left_side_into(b_add_l_mod_2t_in_non_neg_domain)

In [ ]:
prob_b_mod_add_l_eq_1 = register_qubit_born_rule.instantiate(
        {n: t_,
         k: ModAdd(b_, l),
         Psi: Psi_,
         m: m_},
         assumptions=[l_in_domain])

In [ ]:
# recall:
alpha_def

In [ ]:
alpha_def_inst = alpha_def.instantiate(assumptions=[l_in_domain])

In [ ]:
prob_as_alpha_sqrd = alpha_def_inst.sub_left_side_into(prob_b_mod_add_l_eq_1)

Would like to prove that full_domain $\supseteq$ neg_domain and full_domain $\supseteq$ pos_domain, which allows us to later use inclusive_universal_quantification (see about 10 steps below).

In [ ]:
# trivial, but verifying
LessEq(Add(Neg(two_pow_t_minus_one), one), Add(Neg(two_pow_t_minus_one), one)).prove()

In [ ]:
# recall:
eps_upper_bound

In [ ]:
eps_upper_bound_trans_01 = eps_upper_bound.derive_shifted(one)

In [ ]:
eps_upper_bound_trans_02 = eps_upper_bound_trans_01.derive_negated().inner_expr().lhs.distribute()

In [ ]:
Less(Neg(Add(eps, one)), zero).prove(assumptions=[eps_in_domain])

In [ ]:
LessEq(Neg(Add(eps, one)), two_pow_t_minus_one).prove()

In [ ]:
superset_eq(full_domain, neg_domain).prove(assumptions=[eps_in_domain])

In [ ]:
pos_domain

In [ ]:
LessEq(Add(Neg(two_pow_t_minus_one), one), Add(eps, one)).prove(assumptions=[eps_in_domain])

In [ ]:
eps_upper_bound.derive_shifted(one).add_right(one)

In [ ]:
superset_eq(full_domain, pos_domain).prove(assumptions=[eps_in_domain])

Having established that full_domain $\supseteq$ neg_domain and that full_domain $\supseteq$ pos_domain, we can proceed to use inclusive_universal_quantification to create some equalities to then be used for instance_substitution:

In [ ]:
forall_over_neg_domain = Forall(l, prob_as_alpha_sqrd.expr, domain=neg_domain)

In [ ]:
# having trouble using the Forall.conclude_via_domain_inclusion because it doesn't instantiate the
# theorem variable y to the desired instance_param, so building from scratch using the bare
# inclusive_universal_quantification theorem in logic.sets.inclusion
from proveit.logic.sets.inclusion import inclusive_universal_quantification
from proveit import Function, A, B, P
_x_sub = forall_over_neg_domain.instance_param
_P_op = Function(P, _x_sub)
_P_op_sub = forall_over_neg_domain.instance_expr
_A_sub = full_domain
_B_sub = neg_domain
_y_sub = forall_over_neg_domain.instance_param
prob_as_alpha_sqrd_over_neg_domain = inclusive_universal_quantification.instantiate(
        {x: _x_sub,
         _P_op: _P_op_sub,
         A: _A_sub,
         B: _B_sub,
         y: _y_sub},
         assumptions=[eps_in_domain],
         auto_simplify=False).derive_consequent()

In [ ]:
forall_over_pos_domain = Forall(l, prob_as_alpha_sqrd.expr, domain=pos_domain)

In [ ]:
# having trouble using the Forall.conclude_via_domain_inclusion because it doesn't instantiate the
# theorem variable y to the desired instance_param, so building from scratch using the bare
# inclusive_universal_quantification theorem in logic.sets.inclusion
from proveit.logic.sets.inclusion import inclusive_universal_quantification
from proveit import Function, A, B, P
_x_sub = forall_over_pos_domain.instance_param
_P_op = Function(P, _x_sub)
_P_op_sub = forall_over_pos_domain.instance_expr
_A_sub = full_domain
_B_sub = pos_domain
_y_sub = forall_over_pos_domain.instance_param
prob_as_alpha_sqrd_over_pos_domain = inclusive_universal_quantification.instantiate(
        {x: _x_sub,
         _P_op: _P_op_sub,
         A: _A_sub,
         B: _B_sub,
         y: _y_sub},
         assumptions=[eps_in_domain],
         auto_simplify=False).derive_consequent()

In [ ]:
# recall:
fail_sum_4

In [ ]:
neg_domain_sum_eq = fail_sum_4.rhs.operands[0].instance_substitution(
        prob_as_alpha_sqrd_over_neg_domain, assumptions=[eps_in_domain])

In [ ]:
pos_domain_sum_eq = fail_sum_4.rhs.operands[1].instance_substitution(
        prob_as_alpha_sqrd_over_pos_domain, assumptions=[eps_in_domain])

In [ ]:
fail_sum_5 = neg_domain_sum_eq.sub_right_side_into(fail_sum_4)

In [ ]:
fail_sum_6 = pos_domain_sum_eq.sub_right_side_into(fail_sum_5)

In [ ]:
%qed